In [1]:
# Dependencies
import pandas as pd
import numpy as np
from PIL import Image
from __future__ import print_function

In [2]:
# These files are a 'workaround', 'trick', or 'hack' to get around the fact that when Pillow scales 
# a png image, it loses transparency. Invoking these functions on the image before and after a 'resize' 
# will retain transparency. See next cell for example use.

def premultiply(im):
    pixels = im.load()
    for y in range(im.size[1]):
        for x in range(im.size[0]):
            r, g, b, a = pixels[x, y]
            if a != 255:
                r = r * a // 255
                g = g * a // 255
                b = b * a // 255
                pixels[x, y] = (r, g, b, a)

def unmultiply(im):
    pixels = im.load()
    for y in range(im.size[1]):
        for x in range(im.size[0]):
            r, g, b, a = pixels[x, y]
            if a != 255 and a != 0:
                r = 255 if r >= a else 255 * r // a
                g = 255 if g >= a else 255 * g // a
                b = 255 if b >= a else 255 * b // a
                pixels[x, y] = (r, g, b, a)

In [7]:
# For now I am assuming a single CSV file with a facwe image filename in the first column and coordinates
# Other rows in the file are expected to have an accessory image file name in the first column and coordinates in other columns.
# We may want to have separate files for target face and accessories.

imageFile = 'faceFileNoImage.csv'
imagesDF = pd.read_csv(imageFile)
imagesDF

,file_name,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,...,nose_tip_x,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y
0,kaggleFace[0]96x96.jpg,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,...,44.420571,57.066803,61.195308,79.970165,28.614496,77.388992,43.312601,72.935459,43.130707,84.485774
1,dylanOriginal384x384.png,222.000000,179.000000,151.000000,176.000000,207.000000,181.000000,237.000000,181.000000,165.000000,...,188.000000,225.000000,225.000000,248.000000,147.000000,247.000000,187.000000,244.000000,187.000000,281.000000
2,dylanOriginal192x192,111.000000,89.500000,75.500000,88.000000,103.500000,90.500000,118.500000,90.500000,82.500000,...,94.000000,112.500000,112.500000,124.000000,73.500000,123.500000,93.500000,122.000000,93.500000,140.500000
3,dylanOriginal96x96,55.500000,44.750000,37.750000,44.000000,51.750000,45.250000,59.250000,45.250000,41.250000,...,47.000000,56.250000,56.250000,62.000000,36.750000,61.750000,46.750000,61.000000,46.750000,70.250000
4,redSunGlasses457x172.png,322.440300,71.089200,139.639500,71.089200,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Assuming a funciton call will pass in a targetFace filename and an accessory filename

imageDirectoryPath = ''

targetFaceFileName = 'dylanOriginal384x384.png'
targetFaceFilePath = imageDirectoryPath + targetFaceFileName
targetFace = Image.open(targetFaceFilePath)
print(targetFace.format, targetFace.size, targetFace.mode)
targetFace.show()

accessoryFileName = 'redSunGlasses457x172.png'
accessoryFilePath = imageDirectoryPath + accessoryFileName
accessory = Image.open(accessoryFilePath)
print(accessory.format, accessory.size, accessory.mode)
accessory.show()


PNG (384, 384) RGB
PNG (457, 172) RGBA


In [9]:
# Picking the row out of the datatable with the face info
targetRowDF = imagesDF.loc[imagesDF['file_name'] == targetFaceFileName]

In [10]:
# Picking the row out of the datatable with the accessory info
accessoryRowDF = imagesDF.loc[imagesDF['file_name'] == accessoryFileName]

In [11]:
# Comparing the target face eye width (center to center) 
# and the accessory eye width (center to center) to determine the scaling factor to make the accessory fit the face
targetEyeWidth = targetRowDF.iloc[0]["left_eye_center_x"]-targetRowDF.iloc[0]["right_eye_center_x"]
print("face eye width: ",targetEyeWidth)
accessoryEyeWidth = accessoryRowDF.iloc[0]["left_eye_center_x"]-accessoryRowDF.iloc[0]["right_eye_center_x"]
print("accessory eye width: ", accessoryEyeWidth)
scaleFactorAcsy2Face = accessoryEyeWidth/targetEyeWidth
print("Scale factor: " , scaleFactorAcsy2Face)

face eye width:  71.0
accessory eye width:  182.80079999999998
Scale factor:  2.574659154929577


In [12]:
# Calculating the new accessory size and keypoint coordinates
newAcsySize = accessory.size/scaleFactorAcsy2Face
newAcsySize = newAcsySize.astype(int)
newAcsyLeftEyeCenterX = int(accessoryRowDF.iloc[0]["left_eye_center_x"]/scaleFactorAcsy2Face)
newAcsyRightEyeCenterX = int(accessoryRowDF.iloc[0]["right_eye_center_x"]/scaleFactorAcsy2Face)
newAcsyLeftEyeCenterY = int(accessoryRowDF.iloc[0]["left_eye_center_y"]/scaleFactorAcsy2Face)
newAcsyRightEyeCenterY = int(accessoryRowDF.iloc[0]["right_eye_center_y"]/scaleFactorAcsy2Face)
print("new accessory dimensions: ",newAcsySize)

new accessory dimensions:  [177  66]


In [13]:
#Scaling the accessory image file with pre and post processing functions to maintain transparency
premultiply(accessory)
accessory = accessory.resize(newAcsySize)
unmultiply(accessory)
print(accessory.format, accessory.size, accessory.mode)
accessory.show()

None (177, 66) RGBA


In [14]:
# pasting the resized, transparent png image onto the test image
x1 = int(targetRowDF.iloc[0]["right_eye_center_x"])-newAcsyRightEyeCenterX
x2 = x1+newAcsySize[0]
y1 = int(targetRowDF.iloc[0]["right_eye_center_y"])-newAcsyRightEyeCenterY
y2 = y1+newAcsySize[1]

box=(x1,y1,x2,y2)
print(box)
targetFace.paste(accessory, box, accessory)
targetFace.show()

(97, 149, 274, 215)
